In [1]:
import cv2
import numpy as np
import math

In [18]:
# 从text文本中读取特征点
def read_points(path) :
    # Create an array of points.
    points = []
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    return points

#读取所有并对所有的面部特征点进行角度调整
def generate_average_face(path,face_number,landmark_number):
    face_landmarks= np.empty((landmark_number,landmark_number,face_number))
    for i in range(1,face_number+1):
        points_path=path+str(i)+'.txt'
        #读取特征点
        points = read_points(points_path)
        points = np.matrix(points)
        #计算特征点和平均特征点的均值和标准化
        aver_cols = np.mean(points,axis=0)
        stdx=np.std(points[:,0])
        stdy=np.std(points[:,1])
        stdn=((stdx)**2+(stdy)**2)**0.5
        s_points=np.matrix((points[:,:]-aver_cols)/stdn)#特征点标准化
        #根据第一张图片计算夹角并旋转图片
        if(i==1):
            t_s_points=s_points#如果不是第一张图则直接计算夹角
        angle=np.mean(np.mod(np.arctan(t_s_points[:,1]/t_s_points[:,0])-np.arctan(s_points[:,1]/s_points[:,0])+math.pi/2,math.pi)-math.pi/2)
        #根据角度旋转所有特征点
        sin=math.sin(angle)
        cos=math.cos(angle)
        x=s_points[:,0]
        y=s_points[:,1]
        x1=x*cos-y*sin
        y1=x*sin+y*cos
        s_points=np.hstack((x1,y1))
        #重新计算旋转后的特征点
        points=np.matrix(s_points[:,:]*stdn+aver_cols)
        #求特征点的总数
        if i==1:
            sum_points=points
        else:
            sum_points=sum_points+points
    #平均特征点
    average_points=sum_points/face_number
    
    #把第一张面孔morph到大众形状中
    scale=5#设定度量为5
    points_path1=path+'1.txt'
    points=read_points(points_path1)
    points=np.array(points)
    L= np.zeros((landmark_number+3,landmark_number+3))
    for i in range(0,landmark_number):
        for j in range(0,landmark_number):
            if i==j:
                L[i,j]=0
            else:
                di=average_points[i,:]-average_points[j,:]
                di=di.getA()
                r2=np.sum((di*di),axis=1)/(scale**2)#对di进行点乘然后按行求和除以度量的平方
                L[i,j]=r2*math.log(r2)/math.log(10)
          
    L[0:landmark_number,landmark_number]=1;
    L[0:landmark_number,landmark_number+1]=points[:,0];
    L[0:landmark_number,landmark_number+2]=points[:,1];
    L[landmark_number,0:landmark_number]=1
    L[landmark_number+1,0:landmark_number]=np.transpose(points[:,0])
    L[landmark_number+2,0:landmark_number]=np.transpose(points[:,1])
    
    Y= np.zeros((landmark_number+3,2))
    Y[0:landmark_number,0:2]=average_points[:,:]
    LD=L**(-1)
    where_are_inf = np.isinf(LD)
    LD[where_are_inf] = 0
    W=(LD[:,0:2])*Y
    #读取对应图片
    img_path=path+'1.jpg'
    img=cv2.imread(img_path)
    
    x_len=img.shape[0]
    y_len=img.shape[1]
    xa=np.zeros((x_len,y_len))
    ya=np.zeros((x_len,y_len))
    for i in range(x_len):
        xa[i,:]=i
    for j in range(y_len):
        ya[:,j]=j
    xa=xa.astype(np.int)
    ya=ya.astype(np.int)
    xo=W[landmark_number,0]+xa*W[landmark_number+1,0]+ya*W[landmark_number+2,0]
    yo=W[landmark_number,1]+xa*W[landmark_number+1,1]+ya*W[landmark_number+2,1]

    for i in range(landmark_number):
        r2=((xa-points[i,0])*(xa-points[i,0])+(ya-points[i,1])*(ya-points[i,1]))/(scale**2)
        xo=xo+W[i,0]*(r2*np.log(r2))/np.log(10)
        yo=yo+W[i,1]*(r2*np.log(r2))/np.log(10)
        print(r2)#r2数值过大可能是由于图片像素太大导致的
    #fill in the gaps caused by morphing
#     for rgb in range(3):
#         im12_x=int(round((max(xo.flatten()))))
#         im12_y=int(round(max(yo.flatten())))
#         im12=np.zeros((im12_x,im12_y))
#         print(im12.shape)
#         for x in range(x_len):
#             for y in range(y_len):
#                 print(np.argmax(int(round(yo(x,y))),axis=1))
#                 im12(np.argmax(int(round(yo(x,y))),axis=1),np.argmax(int(round(xo(x,y))),axis=1))=img(y,x,rgb)
                
  

In [19]:
point_path='/home/fengchao/桌面/monkeyfacemerge/test/'
generate_average_face(point_path,3,68)

/home/fengchao/anaconda3/envs/face/lib/python3.6/site-packages/ipykernel_launcher.py:73: RuntimeWarning: divide by zero encountered in reciprocal
/home/fengchao/anaconda3/envs/face/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: divide by zero encountered in log
/home/fengchao/anaconda3/envs/face/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: invalid value encountered in multiply
/home/fengchao/anaconda3/envs/face/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in log
/home/fengchao/anaconda3/envs/face/lib/python3.6/site-packages/ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in multiply


[[ 64541.8   64463.76  64385.8  ...,  27514.6   27527.76  27541.  ]
 [ 64476.8   64398.76  64320.8  ...,  27449.6   27462.76  27476.  ]
 [ 64411.88  64333.84  64255.88 ...,  27384.68  27397.84  27411.08]
 ..., 
 [ 44124.8   44046.76  43968.8  ...,   7097.6    7110.76   7124.  ]
 [ 44155.88  44077.84  43999.88 ...,   7128.68   7141.84   7155.08]
 [ 44187.04  44109.    44031.04 ...,   7159.84   7173.     7186.24]]
[[ 79077.16  78986.    78894.92 ...,  27093.16  27093.2   27093.32]
 [ 79011.36  78920.2   78829.12 ...,  27027.36  27027.4   27027.52]
 [ 78945.64  78854.48  78763.4  ...,  26961.64  26961.68  26961.8 ]
 ..., 
 [ 57699.36  57608.2   57517.12 ...,   5715.36   5715.4    5715.52]
 [ 57729.64  57638.48  57547.4  ...,   5745.64   5745.68   5745.8 ]
 [ 57760.    57668.84  57577.76 ...,   5776.     5776.04   5776.16]]
[[ 97266.6   97162.    97057.48 ...,  29961.    29947.6   29934.28]
 [ 97198.72  97094.12  96989.6  ...,  29893.12  29879.72  29866.4 ]
 [ 97130.92  97026.32  96921.8  

[[ 89426.    89355.88  89285.84 ...,  61427.6   61448.68  61469.84]
 [ 89329.16  89259.04  89189.   ...,  61330.76  61351.84  61373.  ]
 [ 89232.4   89162.28  89092.24 ...,  61234.    61255.08  61276.24]
 ..., 
 [ 30769.16  30699.04  30629.   ...,   2770.76   2791.84   2813.  ]
 [ 30768.4   30698.28  30628.24 ...,   2770.     2791.08   2812.24]
 [ 30767.72  30697.6   30627.56 ...,   2769.32   2790.4    2811.56]]
[[ 103382.92  103309.2   103235.56 ...,   71280.52   71298.     71315.56]
 [ 103277.6   103203.88  103130.24 ...,   71175.2    71192.68   71210.24]
 [ 103172.36  103098.64  103025.   ...,   71069.96   71087.44   71105.  ]
 ..., 
 [  34541.6    34467.88   34394.24 ...,    2439.2     2456.68    2474.24]
 [  34532.36   34458.64   34385.   ...,    2429.96    2447.44    2465.  ]
 [  34523.2    34449.48   34375.84 ...,    2420.8     2438.28    2455.84]]
[[ 121031.84  120958.76  120885.76 ...,   89659.04   89677.16   89695.36]
 [ 120913.48  120840.4   120767.4  ...,   89540.68   89558

[[ 103777.    103690.64  103604.36 ...,   57265.     57269.84   57274.76]
 [ 103681.44  103595.08  103508.8  ...,   57169.44   57174.28   57179.2 ]
 [ 103585.96  103499.6   103413.32 ...,   57073.96   57078.8    57083.72]
 ..., 
 [  46657.44   46571.08   46484.8  ...,     145.44     150.28     155.2 ]
 [  46657.96   46571.6    46485.32 ...,     145.96     150.8      155.72]
 [  46658.56   46572.2    46485.92 ...,     146.56     151.4      156.32]]
[[ 95858.92  95772.8   95686.76 ...,  49620.52  49625.6   49630.76]
 [ 95770.    95683.88  95597.84 ...,  49531.6   49536.68  49541.84]
 [ 95681.16  95595.04  95509.   ...,  49442.76  49447.84  49453.  ]
 ..., 
 [ 46714.    46627.88  46541.84 ...,    475.6     480.68    485.84]
 [ 46721.16  46635.04  46549.   ...,    482.76    487.84    493.  ]
 [ 46728.4   46642.28  46556.24 ...,    490.      495.08    500.24]]
[[ 138053.84  137969.    137884.24 ...,   93274.64   93281.     93287.44]
 [ 137931.88  137847.04  137762.28 ...,   93152.68   93159

[[ 171116.8   170982.76  170848.8  ...,   70249.6    70206.76   70164.  ]
 [ 171019.88  170885.84  170751.88 ...,   70152.68   70109.84   70067.08]
 [ 170923.04  170789.    170655.04 ...,   70055.84   70013.     69970.24]
 ..., 
 [ 112363.88  112229.84  112095.88 ...,   11496.68   11453.84   11411.08]
 [ 112363.04  112229.    112095.04 ...,   11495.84   11453.     11410.24]
 [ 112362.28  112228.24  112094.28 ...,   11495.08   11452.24   11409.48]]
[[ 184914.32  184779.4   184644.56 ...,   83043.92   83000.2    82956.56]
 [ 184807.72  184672.8   184537.96 ...,   82937.32   82893.6    82849.96]
 [ 184701.2   184566.28  184431.44 ...,   82830.8    82787.08   82743.44]
 ..., 
 [ 114535.72  114400.8   114265.96 ...,   12665.32   12621.6    12577.96]
 [ 114525.2   114390.28  114255.44 ...,   12654.8    12611.08   12567.44]
 [ 114514.76  114379.84  114245.   ...,   12644.36   12600.64   12557.  ]]
[[ 192914.56  192779.24  192644.   ...,   90588.16   90544.04   90500.  ]
 [ 192802.6   192667.2